In [1]:
import numpy as np
import pandas as pd
# pd.set_option("max_rows", 500, "max_colwidth", 500, "max_columns", 500)

In [2]:
# pd.set_option("max_colwidth", 50, "max_rows", 20, "max_columns", 20, "width", 100)
df = pd.read_csv("../scraping/data/article.csv")
df.head()

,url,date,title,context,category,keyword,tweet_share,quote,main_category,context_
0,https://news.livedoor.com/article/detail/20558...,2021-07-19 19:40:00,安田大サーカス・クロちゃん、新型コロナ感染 団長安田とは「別ルートでの感染と考えられます」,\n お笑いトリオ・安田大サーカスのクロちゃん（44）の所属事務所・松竹芸能は19日、クロち...,ニューストップ<SEP>芸能<SEP>芸能総合,クロちゃん<SEP>安田大サーカス<SEP>芸能ニュース<SEP>新型コロナウイルス,1025,オリコン,芸能,お笑いトリオ ・ 安田大サーカス の クロちゃん （ 44 ） の 所属事務所 ・ 松竹芸能...
1,https://news.livedoor.com/article/detail/20558...,2021-07-19 19:32:00,へずまりゅうに懲役求刑“反省みられない”,\n元ユーチューバー、「へずまりゅう」こと原田将大被告（３０）――「（Ｑ．初公判ですがお気持...,ニューストップ<SEP>国内<SEP>社会,YouTuber<SEP>国内ニュース,6,日テレNEWS24,国内,元 ユーチューバー 、 「 へずまりゅう 」 こと 原田 将大 被告 （ 30 ） ―― 「...
2,https://news.livedoor.com/article/detail/20558...,2021-07-19 20:00:00,【独自】Uターン失敗でパニックに？ 歩道や花壇に次々と突っ込む…ドラレコが捉えた謎の暴走事故,\nスピードを上げ…歩道に突進ドライブレコーダーが捉えたのは、コントロールを失った車がスピー...,ニューストップ<SEP>国内,高齢ドライバーの事故<SEP>交通事故<SEP>国内の事件・事故,3,FNNプライムオンライン,国内,スピード を 上げ … 歩道 に 突進 ドライブレコーダー が 捉え た の は 、 コント...
3,https://news.livedoor.com/article/detail/20558...,2021-07-19 19:32:00,小山田圭吾のいじめ告白記事『Quick Japan』出版元も謝罪 「差別を助長する不適切なもの」,\n ミュージシャンの小山田圭吾（52）が、学生時代に障がい者へのいじめを行っていたと告白し...,ニューストップ<SEP>芸能<SEP>芸能総合,小山田圭吾<SEP>東京五輪(2020),358,オリコン,芸能,ミュージシャン の 小山田圭吾 （ 52 ） が 、 学生時代 に 障がい者 へ の いじめ...
4,https://news.livedoor.com/article/detail/20555...,2021-07-19 12:03:00,韓国 日本大使館幹部処分を首脳会談条件に,\n韓国の文在寅大統領の日本訪問と首脳会談について１９日、韓国側は、不適切な発言を行ったソウ...,ニューストップ<SEP>海外<SEP>海外総合,日韓首脳会談<SEP>日韓関係<SEP>韓国の話題<SEP>海外ニュース,28,日テレNEWS24,海外,韓国 の 文在寅 大統領 の 日本 訪問 と 首脳会談 について 19日 、 韓国側 は 、...


In [3]:
df.context_.str.split(" ").str.len().describe()
# 文のサイズは1024で固定

count     5897.000000
mean       676.425810
std        658.887781
min         17.000000
25%        272.000000
50%        483.000000
75%        840.000000
max      11076.000000
Name: context_, dtype: float64

In [4]:
main_category_dict = {v:i for i,v in enumerate(df.main_category.unique())}
main_category_dict

{'芸能': 0,
 '国内': 1,
 '海外': 2,
 'ライフ総合': 3,
 'IT 経済': 4,
 'スポーツ': 5,
 '車': 6,
 'グルメ': 7,
 'ライフスタイル': 8,
 '恋愛': 9,
 'ファッション・ビューティ': 10}

In [5]:
df["label_"] = df.main_category.map(main_category_dict)

In [6]:
df[["context_", "label_"]].to_csv("./datasets.csv", index=None, header=None)

In [7]:
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F


In [17]:
# 本文フィールド
context_field = torchtext.data.Field(
#    sequential=True,  # Falseなら、tokenizeしない (デフォルト: True)
    use_vocab=True,  # Trueなら、ボキャブラリを使って数値化する
#    init_token="<INIT>",  # 文頭に付与するトークン (デフォルト: None)
#    eos_token="<EOS>",  # 文末に付与するトークン (デフォルト: None)
    fix_length=1024,  # 1文あたりのワード数 (不足する場合はパディング)
#    pad_first=False,  # パディングを最初にいれるかどうか (デフォルト: False)
#    truncate_first=False,  # fix_lengthを超えた場合に先頭から削るかどうか (デフォルト: False -> 後ろから削られる)
#    tokenize=lambda x: x.split(" "),  # トークナイズ処理 (デフォルト: str.split)
#    stop_words=set(stopwords.words("english")),  # 除去するストップワード
#    preprocessing=torchtext.data.Pipeline(preprocessing),  # 後述 (デフォルト: None)
#    postprocessing=None,  # ミニバッチ単位で行う処理 (デフォルト: None)
#    dtype=torch.long,  # データの型 (デフォルト: torch.long)
    batch_first=True,  # ミニバッチの次元を最初に追加するかどうか (デフォルト: False)
#     include_lengths=False,  # パディングした文とあわせて長さを返すかどうか (デフォルト: False)
#     is_target=False  # ラベルフィールドかどうか (デフォルト: False) 
)
# ラベルフィールド
label_field = torchtext.data.Field(
    sequential=False,
    use_vocab=False,
#    is_target=True
)

In [18]:
# TSVファイルからDatasetにロード
dataset = torchtext.data.TabularDataset(
    path="./datasets.csv",
    format="csv", # tsv, csv, json
    fields=[("Context", context_field), ("Label", label_field)],
)

# trainとtestで分割
train_dataset, test_dataset = dataset.split(
    split_ratio=0.9,
)

In [27]:
context_field.build_vocab(train_dataset, min_freq=1)

In [28]:
train_dl = torchtext.data.Iterator(train_dataset, batch_size=128)

In [29]:
test_dl = torchtext.data.Iterator(test_dataset, batch_size=128, train=False, sort=False)

In [30]:
batch = next(iter(train_dl))

In [33]:
batch.Context[0], batch.Label[0]

(tensor([  102, 10842,   101,  ...,     1,     1,     1]), tensor(2))